In [21]:
from dataclasses import dataclass

# Day 1

In [9]:
with open('Advent_2020_Day_1.txt') as f:
    nums = f.read().splitlines()

In [22]:
def check_sum(a: int, b: int) -> bool:
    if a+b == 2020:
        return True
    
@dataclass
class Coord:
    a: int
    b: int

In [46]:
pt = Coord(0,1)

while pt.b <= len(nums)-1 and pt.a <= len(nums)-1:
    if check_sum(nums[pt.a], nums[pt.b]):
        print(pt.a,pt.b)
        break
    else:
        print('checked', pt.a, pt.b)
    #Check if a and b are the same, else add 1
    if pt.a == pt.b:
        pt.b += 1
        # print('1 added to b')
        continue

    #Check if b is at the end, else add 1 to a and reset b
    if pt.b == len(nums) -1:
        pt.a+=1
        pt.b = pt.a + 1
        # print('1 added to a')
        print('b reset')
        continue
    #Else, add one to b
    pt.b += 1
    # print('1 added to b')
    continue

checked 0 1
checked 0 2
checked 0 3
checked 0 4
checked 0 5
checked 0 6
checked 0 7
checked 0 8
checked 0 9
checked 0 10
checked 0 11
checked 0 12
checked 0 13
checked 0 14
checked 0 15
checked 0 16
checked 0 17
checked 0 18
checked 0 19
checked 0 20
checked 0 21
checked 0 22
checked 0 23
checked 0 24
checked 0 25
checked 0 26
checked 0 27
checked 0 28
checked 0 29
checked 0 30
checked 0 31
checked 0 32
checked 0 33
checked 0 34
checked 0 35
checked 0 36
checked 0 37
checked 0 38
checked 0 39
checked 0 40
checked 0 41
checked 0 42
checked 0 43
checked 0 44
checked 0 45
checked 0 46
checked 0 47
checked 0 48
checked 0 49
checked 0 50
checked 0 51
checked 0 52
checked 0 53
checked 0 54
checked 0 55
checked 0 56
checked 0 57
checked 0 58
checked 0 59
checked 0 60
checked 0 61
checked 0 62
checked 0 63
checked 0 64
checked 0 65
checked 0 66
checked 0 67
checked 0 68
checked 0 69
checked 0 70
checked 0 71
checked 0 72
checked 0 73
checked 0 74
checked 0 75
checked 0 76
checked 0 77
checked 